# [GetOldTweets](https://github.com/Mottl/GetOldTweets3) 사용법
- `.tweetCriteria()` 설정 : 날짜 설정 시 마지막 날짜 포함되지 않음에 주의.
- `.tweetManager` 호출 후 `getTweets()` 메소드 실행.

In [ ]:
# install GOT3 package
!pip install GetOldTweets3

# Ver 1 : 날짜 수기 입력

- 날짜 입력 에러 커스텀. 
- `getJsonResponse` 함수 수정.
    - 디버그 옵션 수정 : 디버그 시 헤더 정보, json 정보 전부 출력하지 않도록 변경.
    - HTTP 429 에러: `time.sleep`. 10 -> 5 -> 3 -> 2 -> 1.
    - TimeOut Error: 한 번 더 기다리고 안 되면 pass.
    - Url 에러, Json 파싱 에러: 바로 pass.

In [19]:
# module import
import GetOldTweets3 as got
from bs4 import BeautifulSoup

import sys
import urllib
import json
import datetime
import time
import os
from random import uniform
from tqdm import tqdm
import csv
import pandas as pd

In [20]:
def getJsonResponse(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
    """
    Invoke an HTTP query to Twitter.
    Should not be used as an API function. A static method.
    """
    url = "https://twitter.com/i/search/timeline?"

    if not tweetCriteria.topTweets:
        url += "f=tweets&"

    url += ("vertical=news&q=%s&src=typd&%s"
            "&include_available_features=1&include_entities=1&max_position=%s"
            "&reset_error_state=false")

    urlGetData = ''

    if hasattr(tweetCriteria, 'querySearch'):
        urlGetData += tweetCriteria.querySearch

    if hasattr(tweetCriteria, 'excludeWords'):
        urlGetData += ' -'.join([''] + tweetCriteria.excludeWords)

    if hasattr(tweetCriteria, 'username'):
        if not hasattr(tweetCriteria.username, '__iter__'):
            tweetCriteria.username = [tweetCriteria.username]

        usernames_ = [u.lstrip('@') for u in tweetCriteria.username if u]
        tweetCriteria.username = {u.lower() for u in usernames_ if u}

        usernames = [' from:'+u for u in sorted(tweetCriteria.username)]
        if usernames:
            urlGetData += ' OR'.join(usernames)

    if hasattr(tweetCriteria, 'within'):
        if hasattr(tweetCriteria, 'near'):
            urlGetData += ' near:"%s" within:%s' % (tweetCriteria.near, tweetCriteria.within)
        elif hasattr(tweetCriteria, 'lat') and hasattr(tweetCriteria, 'lon'):
            urlGetData += ' geocode:%f,%f,%s' % (tweetCriteria.lat, tweetCriteria.lon, tweetCriteria.within)

    if hasattr(tweetCriteria, 'since'):
        urlGetData += ' since:' + tweetCriteria.since

    if hasattr(tweetCriteria, 'until'):
        urlGetData += ' until:' + tweetCriteria.until

    if hasattr(tweetCriteria, 'minReplies'):
        urlGetData += ' min_replies:' + tweetCriteria.minReplies

    if hasattr(tweetCriteria, 'minFaves'):
        urlGetData += ' min_faves:' + tweetCriteria.minFaves

    if hasattr(tweetCriteria, 'minRetweets'):
        urlGetData += ' min_retweets:' + tweetCriteria.minRetweets

    if hasattr(tweetCriteria, 'lang'):
        urlLang = 'l=' + tweetCriteria.lang + '&'
    else:
        urlLang = ''
    url = url % (urllib.parse.quote(urlGetData.strip()), urlLang, urllib.parse.quote(refreshCursor))
    useragent = useragent or TweetManager.user_agents[0]

    headers = [
        ('Host', "twitter.com"),
        ('User-Agent', useragent),
        ('Accept', "application/json, text/javascript, */*; q=0.01"),
        ('Accept-Language', "en-US,en;q=0.5"),
        ('X-Requested-With', "XMLHttpRequest"),
        ('Referer', url),
        ('Connection', "keep-alive")
    ]

    if proxy:
        opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
    else:
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
    opener.addheaders = headers

    # 디버그 시 헤더 정보 출력 수정
    if debug:
        print(url)
        # print('\n'.join(h[0]+': '+h[1] for h in headers))

    # HTTP Request 429 에러 방지
    time.sleep(3)

    try:
        response = opener.open(url)
        jsonResponse = response.read()

    except TimeoutError as e:
        print("Timeout error")
        print('sleep 30')
        time.sleep(30)

        # 이 부분 나중에 retry 매직 메소드 알아보기 / 강사님 질문
        try:
            response = opener.open(url)
            jsonResponse = response.read()
        except TimeoutError as e:
            print("Timeout Error again.")
            pass

    except Exception as e:
        print("An error occured during an HTTP request:", str(e))
        print("Error URL:", url)
        print("Try to open in browser: https://twitter.com/search?q=%s&src=typd" % urllib.parse.quote(urlGetData))

        # 이 부분 나중에 확인하기
        print("sleep 30!!")
        pass
        # sys.exit()
        

    try:
        s_json = jsonResponse.decode()
    except:
        print("Invalid response from Twitter")
        print("Error URL:", url)
        pass
        # sys.exit()
    else:
        try:
            dataJson = json.loads(s_json)
        except:
            print("Error parsing JSON: %s" % s_json)
            print("Error URL:", url)
        pass
        # sys.exit()
        
#     if debug:
#         print(s_json)
#         print("---\n")


    return dataJson

In [21]:
# 날짜 다시 입력해야 할 때 에러 발생시키자.
class NotValidEndDateError(Exception):
    def __init__(self):
        super().__init__('마지막 검색 날짜를 다시 설정하십시오.')

In [22]:
# GetOldTweets3 패키지의 setUntil 메소드는 마지막 날짜 포함하지 않으므로, 주의.
def set_crawl_date(start_date, end_date):
    
    start_date = datetime.datetime.strptime(str(start_date), "%Y%m%d")
    end_date = datetime.datetime.strptime(str(end_date), "%Y%m%d") - datetime.timedelta(days=1)
    
    if end_date == start_date:
        raise NotValidEndDateError
    
    else:   
        print("트윗 수집 날짜 설정: {0}부터 {1}까지".format(start_date, end_date))    
        return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")

In [23]:
def crawl_tweets(start_date, end_date, debug=False):    
    got.manager.TweetManager.getJsonResponse = getJsonResponse # 함수에 이 부분 넣어주고
    
    print("========== 트윗 수집 시작: {0} ~ {1} ==========".format(start_date, end_date))
    start_time = time.time()
    tweet_criteria = got.manager.TweetCriteria().setQuerySearch('Elon Musk')\
                                                .setSince(start_date)\
                                                .setUntil(end_date)\
                                                .setLang('en')
    tweets = got.manager.TweetManager.getTweets(tweet_criteria, debug=debug)
    
    elapsed_time = time.time()-start_time
    
    print("수집 완료 : {}".format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    print("총 수집 트윗 개수 : {0}".format(len(tweets)))
    
    return tweets

In [24]:
def get_results(tweet_data):
    results = []
    for tweet in tqdm(tweet_data):
        results.append({'url': tweet.permalink,
                        'date': tweet.date,
                        'text': tweet.text,
                        'user': tweet.username,
                        'mentions': tweet.mentions,
                        'retweets': tweet.retweets,
                        'favorites': tweet.favorites,
                        'hashtags': tweet.hashtags})
        
    return results        

In [25]:
def save_tweets(tweet_lists):
    base_file_dir = "tweets"

    if not os.path.exists(base_file_dir):
        os.makedirs(base_file_dir)
        
    with open(f"{base_file_dir}/tweets_{crawl_start}_{crawl_end}.csv", "a", -1, encoding="utf-8") as f:    
        writer = csv.writer(f)
        writer.writerow(['url', 'date', 'text', 'user', 'mentions', 'retweets', 'favorites', 'hashtags'])        
        for tweet_list in tqdm(tweet_lists):
            writer.writerow(list(tweet_list.values()))
            
    return 

In [ ]:
# 크롤링 진행
crawl_start, crawl_end = set_crawl_date(20150101, 20150103)
tweet_results = crawl_tweets(crawl_start, crawl_end)
tweet_results_lists = get_results(tweet_results)
save_tweets(tweet_results_lists)

트윗 수집 날짜 설정: 2015-01-01 00:00:00부터 2015-01-02 00:00:00까지
========== 트윗 수집 시작: 2015-01-01 ~ 2015-01-02 ==========


# Ver 2

- date_range 추가: 날짜 입력 by 반복문 (날짜 exception class 없음)
- url 설정 부분 간소화: 필요한 부분만 url 돌리도록 설정.
- crawl 함수 변경: 검색어 변경.

In [15]:
# module import
import GetOldTweets3 as got
from bs4 import BeautifulSoup

import sys
import urllib
import json
from time import sleep
import pandas as pd

import datetime
import time
import os
from random import uniform
from tqdm import tqdm
import csv

In [9]:
# GOT 스태틱 메소드 수정
def getJsonResponse(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
    url = "https://twitter.com/i/search/timeline?"

    if not tweetCriteria.topTweets:
        url += "f=tweets&"

    url += ("vertical=news&q=%s&src=typd&%s"
            "&include_available_features=1&include_entities=1&max_position=%s"
            "&reset_error_state=false")

    urlGetData = ''
    
    # url + query search, since, since, until, lang    
    urlGetData += tweetCriteria.querySearch
    urlGetData += ' since:' + tweetCriteria.since
    urlGetData += ' until:' + tweetCriteria.until
    urlLang = 'l=' + tweetCriteria.lang + '&'
    
    # url 설정
    url = url % (urllib.parse.quote(urlGetData.strip()), urlLang, urllib.parse.quote(refreshCursor))
    useragent = useragent or TweetManager.user_agents[0]

    headers = [
        ('Host', "twitter.com"),
        ('User-Agent', useragent),
        ('Accept', "application/json, text/javascript, */*; q=0.01"),
        ('Accept-Language', "en-US,en;q=0.5"),
        ('X-Requested-With', "XMLHttpRequest"),
        ('Referer', url),
        ('Connection', "keep-alive")
    ]

    if proxy:
        opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
    else:
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
    opener.addheaders = headers        
    
    time.sleep(1) # HTTP Request 429 에러 방지.

    ########################################## 에러 핸들링 수정 ##########################################
    
    # TimeOut 에러
    try:
        response = opener.open(url)
        jsonResponse = response.read()
    except TimeoutError as e:
        
        if debug: # 디버그 옵션
            print("에러 url 주소:", url) 
            
        print("Timeout error")
        print("30초 정지")
        time.sleep(30)

        # 한 번 더 시도
        try:
            response = opener.open(url)
            jsonResponse = response.read()
        except TimeoutError as e:
            print("Timeout error again. 패스.")
            pass

    # UrlParse 에러: 다시 시도 X
    except Exception as e:
        if debug: # 디버그 옵션
            print("에러 url 주소:", url)
            
        print("HTTP 요청 오류", str(e))        
        print("브라우저 오픈: https://twitter.com/search?q=%s&src=typd" % urllib.parse.quote(urlGetData))
        print("30초 정지")
        
        pass

    # Json 데이터 오류
    try:
        s_json = jsonResponse.decode()
    except:
        print("올바르지 못한 응답")
        if debug: # 디버그 옵션
            print("에러 url 주소:", url)
        pass
    else:
        try:
            dataJson = json.loads(s_json)
        except: # json 데이터 파싱 오류
            if debug: # 디버그 옵션
                print("에러 url 주소:", url)                
            print("JSON: %s" % s_json)
        pass

    return dataJson

In [10]:
# setUntil : 마지막 날짜 배제되므로 주의.
def set_crawl_date(start_date, freq=1):    
    end_date = start_date + datetime.timedelta(days=freq)
    
    # timestamp to string format
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    # check
    print("트윗 수집 날짜 설정: {0}부터 {1}까지".format(start_date, end_date)) 
    
    return start_date, end_date

In [11]:
def crawl_tweets(start_date, end_date, query, lang='en', debug=True):    
    '''
    query: 검색할 트윗 검색어
    lang: 검색할 트윗 언어
    debug: 설정 시 에러 url 표시
    '''
    
    print("========== 트윗 수집 시작: {0} ~ {1} ==========".format(start_date, end_date))
    start_time = time.time()
    tweet_criteria = got.manager.TweetCriteria().setQuerySearch(query)\
                                                .setSince(start_date)\
                                                .setUntil(end_date)\
                                                .setLang(lang)
    tweets = got.manager.TweetManager.getTweets(tweet_criteria, debug=debug)
    
    elapsed_time = time.time()-start_time
    
    print("수집 완료 : {}".format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    print("총 수집 트윗 개수 : {0}".format(len(tweets)))
    
    return tweets

In [12]:
# got tweet 객체로부터 결과 추출
def get_results(tweet_data):
    results = []
    for tweet in tqdm(tweet_data):
        results.append({'url': tweet.permalink,
                        'date': tweet.date,
                        'text': tweet.text,
                        'user': tweet.username,
                        'mentions': tweet.mentions,
                        'retweets': tweet.retweets,
                        'favorites': tweet.favorites,
                        'hashtags': tweet.hashtags})
    return results

In [13]:
# 추출한 결과 저장
def save_tweets(tweet_lists, base_file_dir="tweets"):
    
    if not os.path.exists(base_file_dir):
        os.makedirs(base_file_dir)
        
    with open(f"{base_file_dir}/tweets_{crawl_start}_{crawl_end}.csv", "a", -1, encoding="utf-8") as f:    
        writer = csv.writer(f)
        writer.writerow(['url', 'date', 'text', 'user', 'mentions', 'retweets', 'favorites', 'hashtags'])        
        for tweet_list in tqdm(tweet_lists):
            writer.writerow(list(tweet_list.values()))
            
    return 

In [18]:
# 크롤링 진행
dateRange = pd.date_range(start='20150101', end='20200601', freq='MS') # freq 변경 가능: 며칠치 데이터 가져올 지 설정.

for date in dateRange:
    crawl_start, crawl_end = set_crawl_date(date)  # freq 변경 가능: 며칠치 데이터 가져올 지 설정.
    tweet_results = crawl_tweets(crawl_start, crawl_end, query='Elon Musk', debug=False)
    tweet_results_lists = get_results(tweet_results)
    save_tweets(tweet_results_lists)

트윗 수집 날짜 설정: 2015-01-01부터 2015-01-02까지
========== 트윗 수집 시작: 2015-01-01 ~ 2015-01-02 ==========


100%|███████████████████████████████████████████████████████████████████████████| 1654/1654 [00:00<00:00, 87261.53it/s]

수집 완료 : 00:02:05
총 수집 트윗 개수 : 1654
트윗 수집 날짜 설정: 2015-02-01부터 2015-02-02까지
========== 트윗 수집 시작: 2015-02-01 ~ 2015-02-02 ==========



100%|█████████████████████████████████████████████████████████████████████████████| 530/530 [00:00<00:00, 75921.49it/s]

수집 완료 : 00:00:44
총 수집 트윗 개수 : 530
트윗 수집 날짜 설정: 2015-03-01부터 2015-03-02까지
========== 트윗 수집 시작: 2015-03-01 ~ 2015-03-02 ==========


...


100%|███████████████████████████████████████████████████████████████████████████| 1180/1180 [00:00<00:00, 65740.57it/s]

수집 완료 : 00:01:29
총 수집 트윗 개수 : 1180
트윗 수집 날짜 설정: 2015-09-01부터 2015-09-02까지
========== 트윗 수집 시작: 2015-09-01 ~ 2015-09-02 ==========



100%|██████████████████████████████████████████████████████████████████████████| 1132/1132 [00:00<00:00, 169964.28it/s]

수집 완료 : 00:01:26
총 수집 트윗 개수 : 1132
트윗 수집 날짜 설정: 2015-10-01부터 2015-10-02까지
========== 트윗 수집 시작: 2015-10-01 ~ 2015-10-02 ==========


KeyboardInterrupt: 